In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F

During this seminar we will implement 2 most confusing but very popular layers - BatchNorm and Dropout along with the most popular architectures such as VGG, Inception, ResNet, ResNeXt, DenseNet, SENet and Mobilenet_v2.

The main goal of this seminar is to make you understand how simple is to create your **arbitrary** neural network in pytorch and to give you some practical experience. PyTorch is very flexible and still simple, so you can implement any idea in just a few lines of code!

Enjoy!

# Layers

## Dropout

In [ ]:
class Dropout(nn.Module):
    def __init__(self, p):
        super().__init__()
        self.p = p
        
    def forward(self, x):
        # there is F.dropout(...) in torch, but let's not use it here
        # use self.training flag
        # TODO(students)
        
        if self.training:
            mask = torch.empty(x.size()).bernoulli(1. - self.p)
            return x * mask / (1. - self.p)
        else:
            return x

## BatchNorm

In [ ]:
# TODO(me) still good realization somewhere
class BatchNorm2D(nn.Module):
    def __init__(self, momentum=0.1):
        super().__init__()
        self.momentum = momentum
        
        
    def forward(self, x):
        # use self.training flag, don't forget to update running mean and variance

## Architectures

## Alexnet

In [ ]:
# source: torchvision
class AlexNet(nn.Module):

    def __init__(self, num_classes=1000):
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

In [ ]:
class VGG(nn.Module):
    def __init__(self, cfg, use_bn=False, num_classes=1000):
        super(VGG, self).__init__()
        self.features = VGG.make_layers(cfg, batch_norm=use_bn)
        self.avgpool = nn.AdaptiveAvgPool2d((7, 7))
        self.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, num_classes),
        )
        if init_weights:
            self._initialize_weights()
    
    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x
    
    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)
    
    cfgs = {
        'vgg11': [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
        'vgg13': [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
        'vgg16': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'],
        'vgg19': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M'],
    }
    
    @static_method
    def make_layers(cfg, batch_norm=False):
        layers = []
        in_channels = 3
        for v in cfg:
            if v == 'M':
                layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
            else:
                conv2d = nn.Conv2d(in_channels, v, kernel_size=3, padding=1)
                if batch_norm:
                    layers += [conv2d, nn.BatchNorm2d(v), nn.ReLU(inplace=True)]
                else:
                    layers += [conv2d, nn.ReLU(inplace=True)]
                in_channels = v
        return nn.Sequential(*layers)

So far we have seen only sequential models which are essentially sequence of conv, pool, nonlinearity and normalization layers. Such architectures are in most cases very stable in training, but typically are slower, heavier and even converge to lower values. Modern architectures are somehow "less sequential", the main ideas are: compute several branches in parallel (inception), use information from previous layers and not only the last one (resnet, densenet).

## Inception
Compute several branches in parallel to ensure larger context

In [ ]:
# source: torchvision
class Inception(nn.Module):
    def __init__(self, in_channels, ch1x1, ch3x3red, ch3x3, ch5x5red, ch5x5, pool_proj,
                 conv_block=None):
        super(Inception, self).__init__()
        if conv_block is None:
            conv_block = BasicConv2d
        # TODO(students): implement all branches based on the picture
        self.branch1 = conv_block(in_channels, ch1x1, kernel_size=1)

        self.branch2 = nn.Sequential(
            conv_block(in_channels, ch3x3red, kernel_size=1),
            conv_block(ch3x3red, ch3x3, kernel_size=3, padding=1)
        )

        self.branch3 = nn.Sequential(
            conv_block(in_channels, ch5x5red, kernel_size=1),
            conv_block(ch5x5red, ch5x5, kernel_size=3, padding=1)
        )

        self.branch4 = nn.Sequential(
            nn.MaxPool2d(kernel_size=3, stride=1, padding=1, ceil_mode=True),
            conv_block(in_channels, pool_proj, kernel_size=1)
        )

    def _forward(self, x):
        branch1 = self.branch1(x)
        branch2 = self.branch2(x)
        branch3 = self.branch3(x)
        branch4 = self.branch4(x)

        outputs = [branch1, branch2, branch3, branch4]
        return outputs

    def forward(self, x):
        outputs = self._forward(x)
        return torch.cat(outputs, 1)

# Resnet

In [1]:
# TODO(me) resnet naive block

# TODO(me) resnet fine block

# TODO(me) wide resnet block

# TODO(me) resneXt block


# Densenet

In [2]:
# TODO(me) densenet

# SENet

In [3]:
# TODO(me) SE-module

# Mobilenet v2

In [4]:
# TODO(me) mobilenet v2

# I can write models and modules and so what?
How to compare your models and modules?
- final network performance (your target metrics)
- inference speed (in milliseconds on specific device)
- FLOPS (rough approximation of real performance)
- number of parameters (== weight of your model)
- speed of training (how many epochs/time your model need to converge)

In [5]:
# TODO(me) architectures chart

## Accuracy vs FLOPS

In [6]:
# TODO(me) architectures chart

## Inference speed

In [7]:
# Disclaimer - on colab comparation may be very unfair

# TODO(me) compare inference speed of torchvision architectures
# [optional] TODO(me) about torchscript

## Number of parameters

In [ ]:
# TODO(me) calculate number of parameters for torchvision arcitectures and written modules

## Convergence speed
Add batch normalization layers to speed up model convergence (up to 10 times speed up in average). Essentially convolution and batch normalization both apply a linear transformation so you can "fuse" batch normalization into the neighbouring convolutional layer, modifying it's weights which will speed up your model inference.

## Practical tips how to solve arbitrary DL problem
1. start from a reasonable baseline (e.g. resnet50)
2. improve target metrics as much as you can (your model may be big or slow, but you will now how good in theory your results should be)
3. if necessary, compress your architecture and try smaller ones. Archieve reasonable speed and try to retain as much accuracy as you can